# Semantic Network

In [1]:
import pprint
from rdflib import Graph, URIRef, Literal,RDF,RDFS
from rdflib.namespace import FOAF

import numpy as np
import pandas as pd


In [2]:
class Node:
    Poi_ID = ''
    name = ''
    Uri = ''
    Pref = 0
    Activate = 0.0
    FactorCtx = {}
    Context = 0
    conf = 0.0
    alfa = 0.1
    
    def __init__(self,Uri):
        self.Uri = Uri
    
    def updateContext(self,nContextAncestros):
        sum_ = 0
        sum_factor = 0
        for key, value in self.FactorCtx.items():
            sum_factor = self.FactorCtx[key]
            sum_ += 1
        if sum_factor== 0:
            self.Context = 0.1
        else:
            self.Context = sum_factor/sum_
            
            
        sum_contex = 0
        for pref_ in nContextAncestros:
            sum_contex += 1
        self.Context = (sum_contex + self.Context)/ (len(nContextAncestros) + 1)
        
    def updatePref(self,nprefAncestros):
        sum_ = 0
        sum_pref = 0
        for pref_ in nprefAncestros:
            pref_ += 1
            sum_ += 1
        conf = conf - 0.1
        self.Pref = sum_pref/sum_-conf
        
        
    def pprint_(self):
        print('Uri', self.Uri)
        print('Pref',self.Pref)
        print('Poi_ID',self.Poi_ID)
        print('Activate',self.Activate)
        print('FactorCtx',self.FactorCtx)
        print('Context',self.Context)
        print('Conf',self.conf)
            

In [3]:
def format_str(str_):
        idx = str_.find(':')
        return str_[idx+1:]


In [4]:
def format_str_num(str_):
        idx = str_.find('#')
        return str_[idx+1:]

In [5]:
class SemanticNetwork:
    uidUser = ''
    NodeObjPOI = {}
    graph = Graph()
    classPr = []
    Subclass = {}
    name_id = {}
    ChildenOf ={}
    
    def __init__(self,uidUser):
        self.uidUser = uidUser
        
    def updatePOIs(self, arrayUri):
        cont = 1
        for uri in arrayUri:
            #print("->",format_str(uri))
            poiUri = URIRef(uri)
            node_= Node(uri)
            node_.name = format_str_num(uri)
            node_.conf = 1
            node_.Poi_ID = f"{cont}p"
            #self.graph.parse(poiUri)
            self.NodeObjPOI[node_.Poi_ID] = node_
            self.name_id[format_str_num(uri)] = f"{cont}p"
            cont+=1
        path = '/home/yessica/Documentos/2021/ProyectoTesis/SR/SRTourism/ontology.ttl'
        self.graph.parse(path,format = 'ttl')
        
        
    def printGraph(self):
        graphSerialize = self.graph.serialize(format="xml")

        #print(g.serialize())

        for subj, pred, obj in self.graph:

            print('->subj ', subj)
            print('->pred ', pred)
            print('->obj ', obj)
            print('\n')
            
    def updatePref(self, arrayPref):
        for key,value in arrayPref.items():
            self.NodeObjPOI[key].Pref = value
            
    def updateContext(self, arrayContext):
        for key,context in arrayContext.items():
            self.NodeObjPOI[key].FactorCtx = context
        
    def spreadingActivation(self):
        for subj, pred, obj in self.graph:
            for POIsk,POINode in self.NodeObjPOI.items():
                uri = POINode.Uri
                if subj == URIRef(uri) and type(obj)==URIRef:
                    subjStr = subj.n3(self.graph.namespace_manager)
                    subj = format_str(subjStr)
                    
                    predStr = pred.n3(self.graph.namespace_manager)
                    pred = format_str(predStr)
                    
                    objStr = obj.n3(self.graph.namespace_manager)
                    obj = format_str(objStr)
                    
                    if pred == 'type':
                        self.classPr.append(subj) #subj self.name_id[subj]
                        
                    elif pred == 'subClassOf':
                        if obj not in self.Subclass:
                            self.Subclass[obj]=[]
                        self.Subclass[obj].append(subj)
                        
                        if subj not in self.ChildenOf:
                            self.ChildenOf[subj]=[]
                        self.ChildenOf[subj].append(obj)
                        
    def propagationPref(self,node_, nodeAncesters):
        sumPrefc = 0
        confC = 0 
        if nodeAncesters.Pref == 0 and node_.Pref>0:
            sumPrefc = node_.Pref * node_.conf
            confC += node_.conf
            nodeAncesters.Pref = sumPrefc/confC
            nodeAncesters.conf = ((node_.conf - node_.alfa )+ nodeAncesters.conf)/2
    
            
    def calculateActivate(self, node_, arrayContext):
        context = arrayContext
        factCtx = node_.FactorCtx
        sumFactor = 0
        nfactor = 0

        for key in factCtx.keys():
            if context[key]== 'any':
                sumFactor += 1
                nfactor += 1
            elif context[key] in factCtx[key]:
                sumFactor += 2
                nfactor += 1
            else:
                sumFactor += 0
                nfactor += 1
            #print('sumFactor :',sumFactor)

        node_.Activate = sumFactor/nfactor
        
    def spreadingActivationPropagation(self):
        for key in self.Subclass.keys():
            if key not in self.name_id:
                idPlace =  f"{len(semanticNet.name_id)+1}p"
                self.name_id[key] = idPlace
                node_= Node('')
                node_.name = key
                node_.Poi_ID = idPlace
                self.NodeObjPOI[node_.Poi_ID] = node_
        
        for key in self.ChildenOf:
            id__ = self.name_id[key]
            node_ = self.NodeObjPOI[id__]
            ancestors = []
            
            arrayContext = {
                    'weather':'rainy',
                    'date':'any',
                    'time':'afternoon',
                    'summer':'any'
                    }
            if node_.Activate <= 0:
                self.calculateActivate( node_, arrayContext)

            for father in self.ChildenOf[key]:
                id_ = self.name_id[father]
                node = self.NodeObjPOI[id_]
                self.propagationPref(node_,node)
                node.Activate = (node_.Activate + node.Activate)/2
    
            
        
                        
                        

In [6]:
semanticNet = SemanticNetwork('id1')
arrUri = [  "https://www.datatourisme.gouv.fr/ontology/core#Museum",
            "https://www.datatourisme.gouv.fr/ontology/core#InterpretationCentre",
            "https://www.datatourisme.gouv.fr/ontology/core#Library",
            "https://www.datatourisme.gouv.fr/ontology/core#ParkAndGarden",
            "https://www.datatourisme.gouv.fr/ontology/core#ArcheologicalSite",
            "https://www.datatourisme.gouv.fr/ontology/core#ReligiousSite",
            "https://www.datatourisme.gouv.fr/ontology/core#RemarkableBuilding",
            "https://www.datatourisme.gouv.fr/ontology/core#CityHeritage",
            "https://www.datatourisme.gouv.fr/ontology/core#DefenceSite",
            "https://www.datatourisme.gouv.fr/ontology/core#RemembranceSite",
            "https://www.datatourisme.gouv.fr/ontology/core#TechnicalHeritage",
            "https://www.datatourisme.gouv.fr/ontology/core#FoodEstablishment",
            "https://www.datatourisme.gouv.fr/ontology/core#NaturalHeritage",
            "https://www.datatourisme.gouv.fr/ontology/core#SportsAndLeisurePlace",
            "https://www.datatourisme.gouv.fr/ontology/core#Store"]


In [7]:
semanticNet.updatePOIs(arrUri)
arrPOIs = semanticNet.NodeObjPOI

In [8]:
arrayPref = {'1p':1.00,      '2p':1.394117647, '3p': 1.029411765, '4p':1.082352941, '5p':1.488235294, 
             '6p':1.064705882,'7p':1.517647059, '8p': 0.7882352941,'9p':0.1941176471,'10p': 0.8705882353,
             '11p':1.311764706,'12p':0.8294117647,'13p':1.029411765,'14p': 1.423529412,'15p': 1.029411765
            }

semanticNet.updatePref(arrayPref)

In [9]:
arrayContext = {'1p':{
                    'weather':['rainy'],
                    'date':['workday'],
                    'time':['morning','afternoon'],
                    'summer':['any']   
                    },     
                '2p':{
                    'weather':['rainy'],
                    'date':['workday','weekend'],
                    'time':['morning','afternoon'],
                    'summer':['any']   
                    }, 
                '3p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }, 
                '4p':{
                    'weather':['rainy'],
                    'date':['workday','weekend'],
                    'time':['morning','afternoon'],
                    'summer':['any'] },  
                '5p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }, 
                '6p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
             '7p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }, 
             '8p': {
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '9p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '10p': {
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '11p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '12p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '13p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '14p':{
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] },
            '15p': {
                    'weather':['sunny'],
                    'date':['workday'],
                    'time':['morning','afternoon','night'],
                    'summer':['any'] }
            }

In [10]:
semanticNet.updateContext(arrayContext)

In [11]:
semanticNet.spreadingActivation()


In [12]:
semanticNet.spreadingActivationPropagation()

# Extraer la relacion de pref y activacion(context) de un Id_User y POIs 

In [15]:
semanticNet.name_id
    

{'Museum': '1p',
 'InterpretationCentre': '2p',
 'Library': '3p',
 'ParkAndGarden': '4p',
 'ArcheologicalSite': '5p',
 'ReligiousSite': '6p',
 'RemarkableBuilding': '7p',
 'CityHeritage': '8p',
 'DefenceSite': '9p',
 'RemembranceSite': '10p',
 'TechnicalHeritage': '11p',
 'FoodEstablishment': '12p',
 'NaturalHeritage': '13p',
 'SportsAndLeisurePlace': '14p',
 'Store': '15p',
 'CulturalSite': '16p',
 'PlaceOfInterest': '17p',
 '//schema.org/LocalBusiness>': '18p',
 '//schema.org/FoodEstablishment>': '19p'}

In [16]:
def relation(semanticNet):
    relation = []
    w1 = 0.3
    w2 = 0.3
    for PoiId in semanticNet.NodeObjPOI:
        rating = arrPOIs[PoiId].Pref*w1 + arrPOIs[PoiId].Activate*w2
        relation.append([semanticNet.uidUser,
                 PoiId,
                 round(arrPOIs[PoiId].Pref, 2),
                 round(arrPOIs[PoiId].Activate, 2),
                 round(rating, 2)
                ])
    return relation

    

In [17]:
relation = relation(semanticNet)
relation

array = np.array(relation)
df = pd.DataFrame(array, columns = ['Id','POIid','Pref','Activate','rating'])

print(df)

     Id POIid  Pref Activate rating
0   id1    1p   1.0      1.5   0.75
1   id1    2p  1.39      1.5   0.87
2   id1    3p  1.03      1.0   0.61
3   id1    4p  1.08      1.5   0.77
4   id1    5p  1.49      1.0   0.75
5   id1    6p  1.06      1.0   0.62
6   id1    7p  1.52      1.0   0.76
7   id1    8p  0.79      1.0   0.54
8   id1    9p  0.19      1.0   0.36
9   id1   10p  0.87      1.0   0.56
10  id1   11p  1.31      1.0   0.69
11  id1   12p  0.83      1.0   0.55
12  id1   13p  1.03      1.0   0.61
13  id1   14p  1.42      1.0   0.73
14  id1   15p  1.03      1.0   0.61
15  id1   16p   1.0     1.06   0.62
16  id1   17p  1.03     0.94   0.59
17  id1   18p   1.0     1.19   0.66
18  id1   19p  0.83      0.5    0.4


In [91]:
df.to_csv('../mod_recomendacion/datasetPropuesta/test_individual.csv') 

# Test Movilens 100K


In [46]:
def rmse_(predictions, targets):
    a = np.sqrt(((predictions - targets) ** 2).mean())
    return a

In [47]:
path_test = '../mod_recomendacion/datasetPropuesta/test_movilensTest.csv'
dataset = pd.read_csv(path_test)

In [48]:
datasetTest = dataset[["preference", "activation"]]
datasetTest

,preference,activation
0,0.111111,0.444444
1,0.000000,0.888889
2,0.555556,0.777778
3,0.888889,0.000000
4,0.888889,0.222222
...,...,...
19995,0.444444,0.222222
19996,1.000000,0.222222
19997,1.000000,0.222222
19998,0.666667,0.444444


In [49]:
rating = dataset[["rating"]]
rating

,rating
0,1.00
1,0.50
2,1.00
3,1.00
4,0.50
...,...
19995,0.75
19996,0.75
19997,0.50
19998,0.50


In [51]:
wp = 0
wc = 0

matriz = []
while wp <= 1:
    array = []
    while wc <= 1:
        wc +=0.1
        dataset1 = datasetTest["preference"]*wp+ datasetTest["activation"]*wc
        rmse = rmse_([dataset1],rating)
        array.append(rmse)
    matriz.append(array)
    wp +=0.1
    wc = 0
    

In [52]:
len(matriz[0])


11

In [53]:
arrayWp = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
arrayWc = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
matriz

tabla_accuraccy = pd.DataFrame(matriz, columns=arrayWc, index=arrayWp)
tabla_accuraccy

,0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0
0.0,rating 0.652462 dtype: float64,rating 0.610959 dtype: float64,rating 0.57254 dtype: float64,rating 0.537865 dtype: float64,rating 0.507701 dtype: float64,rating 0.482894 dtype: float64,rating 0.464305 dtype: float64,rating 0.452699 dtype: float64,rating 0.44862 dtype: float64,rating 0.45227 dtype: float64,rating 0.463467 dtype: float64
0.1,rating 0.608901 dtype: float64,rating 0.568601 dtype: float64,rating 0.531807 dtype: float64,rating 0.499295 dtype: float64,rating 0.471951 dtype: float64,rating 0.450717 dtype: float64,rating 0.436485 dtype: float64,rating 0.42995 dtype: float64,rating 0.431464 dtype: float64,rating 0.440943 dtype: float64,rating 0.457892 dtype: float64
0.2,rating 0.568227 dtype: float64,rating 0.529537 dtype: float64,rating 0.494875 dtype: float64,rating 0.465144 dtype: float64,rating 0.441342 dtype: float64,rating 0.424466 dtype: float64,rating 0.415361 dtype: float64,rating 0.414541 dtype: float64,rating 0.422054 dtype: float64,rating 0.437469 dtype: float64,rating 0.459995 dtype: float64
0.3,rating 0.531102 dtype: float64,rating 0.494548 dtype: float64,rating 0.462656 dtype: float64,rating 0.436451 dtype: float64,rating 0.417005 dtype: float64,rating 0.405294 dtype: float64,rating 0.401992 dtype: float64,rating 0.407306 dtype: float64,rating 0.420908 dtype: float64,rating 0.442035 dtype: float64,rating 0.469672 dtype: float64
0.4,rating 0.498321 dtype: float64,rating 0.464556 dtype: float64,rating 0.436196 dtype: float64,rating 0.41435 dtype: float64,rating 0.400089 dtype: float64,rating 0.394235 dtype: float64,rating 0.397161 dtype: float64,rating 0.408679 dtype: float64,rating 0.428094 dtype: float64,rating 0.454397 dtype: float64,rating 0.486471 dtype: float64
0.5,rating 0.470791 dtype: float64,rating 0.440583 dtype: float64,rating 0.416592 dtype: float64,rating 0.399937 dtype: float64,rating 0.391555 dtype: float64,rating 0.391977 dtype: float64,rating 0.401177 dtype: float64,rating 0.418575 dtype: float64,rating 0.443207 dtype: float64,rating 0.473946 dtype: float64,rating 0.509689 dtype: float64
0.6,rating 0.449479 dtype: float64,rating 0.423653 dtype: float64,rating 0.404843 dtype: float64,rating 0.394054 dtype: float64,rating 0.391951 dtype: float64,rating 0.39867 dtype: float64,rating 0.413781 dtype: float64,rating 0.436415 dtype: float64,rating 0.465474 dtype: float64,rating 0.49984 dtype: float64,rating 0.538497 dtype: float64
0.7,rating 0.435298 dtype: float64,rating 0.414628 dtype: float64,rating 0.401638 dtype: float64,rating 0.397083 dtype: float64,rating 0.401251 dtype: float64,rating 0.413878 dtype: float64,rating 0.434228 dtype: float64,rating 0.461278 dtype: float64,rating 0.49393 dtype: float64,rating 0.531151 dtype: float64,rating 0.57205 dtype: float64
0.8,rating 0.428958 dtype: float64,rating 0.414025 dtype: float64,rating 0.407179 dtype: float64,rating 0.408824 dtype: float64,rating 0.418863 dtype: float64,rating 0.436715 dtype: float64,rating 0.461475 dtype: float64,rating 0.492101 dtype: float64,rating 0.527573 dtype: float64,rating 0.566982 dtype: float64,rating 0.609565 dtype: float64
0.9,rating 0.430803 dtype: float64,rating 0.421882 dtype: float64,rating 0.421121 dtype: float64,rating 0.428563 dtype: float64,rating 0.443797 dtype: float64,rating 0.466058 dtype: float64,rating 0.494399 dtype: float64,rating 0.527841 dtype: float64,rating 0.565479 dtype: float64,rating 0.606534 dtype: float64,rating 0.650357 dtype: float64
